In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [3]:
file_path = '/content/pcos_dataset_2000_rows (1).csv'

df = pd.read_csv(file_path)

In [12]:
df.sample(5)

,Age,Weight_kg,Height_cm,Irregular_Periods,Acne,Hair_Growth,Hair_Loss,Weight_Gain,Stress_Level,Physical_Activity,Family_History_PCOS,Blood_Sugar_mg_dl,PCOS_Risk
1869,31,47,148,1.0,1.0,Yes,No,No,Medium,Low,Yes,122,Low
297,38,57,171,1.0,2.0,Yes,Yes,No,Medium,High,Yes,188,Medium
105,29,46,175,0.0,1.0,Yes,No,No,High,Low,Yes,107,Medium
1941,17,103,151,0.0,1.0,Yes,No,No,Low,Moderate,Yes,116,Low
1246,37,97,178,1.0,1.0,Yes,No,No,Medium,Moderate,Yes,146,High


In [5]:
df['PCOS_Risk'].value_counts()

,count
PCOS_Risk,
Medium,705
Low,651
High,644


In [6]:
df['Acne'] = df['Acne'].fillna('unknown')

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
ohe = OneHotEncoder(sparse_output = False)
le = LabelEncoder()

In [9]:
df['Irregular_Periods'] = ohe.fit_transform(df[['Irregular_Periods']])

In [10]:
df['Acne'].value_counts()

,count
Acne,
Mild,691
unknown,660
Severe,649


In [11]:
acne_categories = [["unknown", "Mild", "Severe"]]

acne_encoder = OrdinalEncoder(categories=acne_categories)

df[["Acne"]] = acne_encoder.fit_transform(df[["Acne"]])

In [13]:
df['Hair_Growth'] = ohe.fit_transform(df[['Hair_Growth']])
df['Hair_Loss'] = ohe.fit_transform(df[['Hair_Loss']])
df['Weight_Gain'] = ohe.fit_transform(df[['Weight_Gain']])
df['Family_History_PCOS'] = ohe.fit_transform(df[['Family_History_PCOS']])

In [14]:
ordinal_categories = [
    ["Low", "Medium", "High"],
    ["Low", "Moderate", "High"]
]

In [15]:
ordinal_encoder = OrdinalEncoder(categories=ordinal_categories)

df[["Stress_Level", "Physical_Activity"]] = ordinal_encoder.fit_transform(
    df[["Stress_Level", "Physical_Activity"]]
)

In [16]:
df.sample(2)

,Age,Weight_kg,Height_cm,Irregular_Periods,Acne,Hair_Growth,Hair_Loss,Weight_Gain,Stress_Level,Physical_Activity,Family_History_PCOS,Blood_Sugar_mg_dl,PCOS_Risk
1740,36,45,146,1.0,2.0,1.0,0.0,1.0,0.0,1.0,1.0,83,Medium
1933,31,50,165,1.0,1.0,0.0,1.0,0.0,0.0,2.0,1.0,158,Low


In [17]:
df['PCOS_Risk'] = le.fit_transform(df['PCOS_Risk'])

In [18]:
df.sample(3)

,Age,Weight_kg,Height_cm,Irregular_Periods,Acne,Hair_Growth,Hair_Loss,Weight_Gain,Stress_Level,Physical_Activity,Family_History_PCOS,Blood_Sugar_mg_dl,PCOS_Risk
608,17,83,167,0.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,71,2
439,19,70,157,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,158,2
1109,17,78,162,1.0,1.0,0.0,1.0,0.0,2.0,1.0,1.0,128,1


In [19]:
#data splitting
X = df.drop('PCOS_Risk',axis=1)
y = df['PCOS_Risk']

In [20]:
#train test splitting
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [21]:
#preprocessing Scaling
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [22]:
#first ANN model
PC11 = Sequential()

PC11.add(Dense(32, activation="relu", input_shape=(X_train.shape[1],)))
PC11.add(Dropout(0.2))

PC11.add(Dense(16, activation="relu"))
PC11.add(Dropout(0.2))

PC11.add(Dense(3, activation="softmax"))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
PC11.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [25]:
history_PCOS11 = PC11.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=32,
    verbose=1,
    validation_data=(X_test, y_test)
)

Epoch 1/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.3175 - loss: 16.2460 - val_accuracy: 0.3200 - val_loss: 2.1513
Epoch 2/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3388 - loss: 7.0369 - val_accuracy: 0.3550 - val_loss: 1.9307
Epoch 3/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3416 - loss: 3.8490 - val_accuracy: 0.3875 - val_loss: 1.2879
Epoch 4/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3317 - loss: 2.5671 - val_accuracy: 0.3275 - val_loss: 1.1357
Epoch 5/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3365 - loss: 1.7678 - val_accuracy: 0.3300 - val_loss: 1.1067
Epoch 6/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3125 - loss: 1.5521 - val_accuracy: 0.3325 - val_loss: 1.1063
Epoch 7/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3476 - loss: 1.2787 - val_accuracy: 0.3375 - val_loss: 1.1003
Epoch 8/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3595 - loss: 1.3138 - val_accuracy: 0.3400 - val_los

In [26]:
#building a rfc model
from sklearn.ensemble import RandomForestClassifier
PC12 = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    random_state=42,
    class_weight="balanced"
)

In [27]:
PC12.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=200,
                       random_state=42)

In [28]:
y_pred = PC12.predict(X_test)

# print("Classification Report:")
# print(classification_report(y_test, y_pred))

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.3275
